<a href="https://colab.research.google.com/github/Shushukang/NLP/blob/main/%EC%8B%A4%EC%8A%B511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문장 유사도를 이용한 ChatBot 개발

In [ ]:
### 필요한 라이브러리 설치
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.7 MB/s eta 0:00:00


## 데이터 불러오기

In [ ]:
### 필요한 라이브러리 임폴트
import pandas as pd

In [ ]:
### chatbot 생성형 데이터 불러오기

# 파일 경로 설정
file_path = '/content/drive/MyDrive/딥러닝/ChatBotData.csv'

# DataFrame 생성
df = pd.read_csv(file_path)

# 결과 확인하기
print(df)

                             Q                         A  label
0                       12시 땡!                하루가 또 가네요.      0
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0
4                      PPL 심하네                눈살이 찌푸려지죠.      0
...                        ...                       ...    ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2
11820              흑기사 해주는 짝남.                    설렜겠어요.      2
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2

[11823 rows x 3 columns]


In [ ]:
### 각 컬럼 별 누락 데이터의 수 확인
print(df.isnull().sum())

Q        0
A        0
label    0
dtype: int64


## 문장 임베딩 벡터 생성

In [ ]:
### 함수 임폴트
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
### 사전 학습된 한국어 모델 다운로드
model_name='ddobokki/klue-roberta-base-nli-sts'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 모두 선택 → control + / = 전체 주석 처리
### 질문 문장에 대한 임베딩 벡터 생성

# Q 컬럼 인덱싱 - Q 컬럼을 대상으로 한다는 일종의 명시
sentences = df.loc[:, 'Q']

# 생성된 임베딩 벡터 저장 → 리스트 생성
embeddings = []

# for문 + model.encode(sentence) + append(embedding) 함수 실행
for sentence in sentences:
  # 질문 문장 1개에 대한 임베딩 벡터 생성
  sent_embedding = model.encode(sentence)
  # 임베딩 결과 저장
  embeddings.append(sent_embedding)

# 결과 확인하기
# 성분 원소로서 임베딩 벡터가 리스트 안에 있어야 함
print(f'리스트의 성분 원소로서 임베딩 벡터의 개수 = {len(embeddings)}개')
print('-'*80)
print(f'첫 번째 질문 문장에 대한 임베딩 벡터 : \n{embeddings[0]}')

In [ ]:
### 저장된 임베딩 벡터의 리스트 불러오기
# list → 파일 저장/불러오기 : pickle 사용(확장자 p)

# 필요한 라이브러리 임폴트
import pickle

# 파일 경로 설정
file_path = '/content/drive/MyDrive/딥러닝/chat_embedding.p'

# pickle 파일 불러오기 ; 암기보다 필요할 때 검색해서 사용
with open(file_path, 'rb') as f:
    loaded_embeddings = pickle.load(f)

# 결과 확인하기
print(f'첫 번째 질문 문장에 대한 임베딩 벡터 : \n{loaded_embeddings[0]}')

첫 번째 질문 문장에 대한 임베딩 벡터 : 
[-1.38743356e-01  1.51100559e-02  5.28436184e-01  7.21492112e-01
 -3.91226649e-01 -7.02952206e-01 -8.01476181e-01  4.91262108e-01
  2.17923969e-02  3.80374342e-01 -6.92348778e-01 -1.11402713e-01
  2.16700718e-01  3.51305455e-01  2.36636266e-01  6.80069029e-01
  1.71046928e-01 -5.12855232e-01 -1.09594338e-01  6.90379366e-02
  2.15588361e-02 -7.99719989e-01 -1.68676987e-01  6.13967478e-01
  3.97407740e-01  4.25234884e-01  5.24639748e-02  2.25223616e-01
 -3.69577527e-01 -3.86964232e-01 -6.03432000e-01  4.99111861e-01
 -1.14106417e-01 -1.77366063e-01  1.42669812e-01 -5.48125327e-01
  4.89641666e-01 -2.73698658e-01  5.87849677e-01  3.24973583e-01
  5.86491376e-02  7.93394744e-01 -1.63209900e-01  6.36754557e-02
 -2.92038709e-01 -4.99974638e-01  2.44824648e-01 -4.05068994e-01
 -2.36140907e-01  2.71784455e-01  7.33441591e-01  1.45673631e-02
 -6.05235100e-01  2.02962831e-01 -3.20214510e-01 -1.98001623e-01
 -6.00059986e-01 -6.08389795e-01 -6.73229277e-01 -3.94299120e-01


## ChatBot 데이터 생성
- 새로운 embedding 컬럼 생성
- 질문에 대한 문장 임베딩 벡터 저장

In [ ]:
# 새로운 컬럼 생성 → 질문 문장에 대한 임베딩 벡터 저장s
df.loc[:, 'embedding'] = loaded_embeddings

# 결과 확인하기
print(f'전체 데이터프레임 확인 : \n{df}')

전체 데이터프레임 확인 : 
                             Q                         A  label  \
0                       12시 땡!                하루가 또 가네요.      0   
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0   
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0   
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0   
4                      PPL 심하네                눈살이 찌푸려지죠.      0   
...                        ...                       ...    ...   
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2   
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2   
11820              흑기사 해주는 짝남.                    설렜겠어요.      2   
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2   
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2   

                                               embedding  
0      [-0.13874336, 0.015110056, 0.5284362, 0.721492...  
1      [0.081922516, 0.30836922, 0.27681017, -0.06013...  
2 

In [ ]:
### unpacking - 전달 받는 데이터 수와 일치해야 함
num1,num2, num3 = [10, 20, 30]

## 사용자 기능 함수 생성

In [ ]:
### 두 문장의 텍스트 유사도 계산 함수 정의
# 직접 암기보다 인터넷 검색을 통해 확인 가능 ex.numpy 1차원 배열 코사인 유사도 계산
# 인공지능(Gemini, 챗GPT 등)을 이용하여 'numpy 1차원 배열을 이용해서 코사인 유사도를 계산할 수 있는 코드를 작성해 주세요'로 확인 가능

# 필요한 함수 임폴트
from numpy import dot
from numpy.linalg import norm


# 사용자 정의 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))

In [ ]:
### 사용자 질문에 대한 응답 구현 함수 정의 → 샘플을 이용한 테스트
'''
1. 사용자의 질문이 들어오면 해당 질문에 대한 문장 임베딩 값을 구한다.
2. 사용자 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 컬럼(df.loc[:, 'embedding'])에 저장해 둔 질문 샘플들의 문장 임베딩 값들을 이용, 코사인 유사도를 계산
    --> 코사인 유사도 값을 저장하는 컬럼(df.loc[:, 'score'])을 생성하고 값을 저장
3. 코사인 유사도 값이 가장 높은 질문 샘플을 찾는다.
4. 해당 질문 샘플과 짝이 되는 답변 샘플을 반환한다.
'''

# 사용자 질문 생성
question = '배가 고파요'

# 사용자 질문 → 임베딩 벡터 생성
user_embedding = model.encode(question)

# 코사인 유사도 계산 (11823개(embedding)와 1개(question)의 코사인 유사도 계산 → 값을 scores에 저장 → df에 score 컬럼 생성(최대값 추출))
scores = []
for embedding in df.loc[:, 'embedding']:
  score = cos_sim(user_embedding, embedding)
  scores.append(score)

print(f'사용자 질문과 챗봇 데이터의 질문 간의 코사인 유사도 : \n{scores}')

print('-'*80)

# score 컬럼 생성 → 코사인 유사도 값 저장
df.loc[:,'score'] = scores
print(f'코사인 유사도 계산의 결과 : \n{df}')

print('-'*80)

# score 컬럼 → 값이 최대인 행 인덱스 추출(pandas.series.idxmax())
idx = df.loc[:,'score'].idxmax()
print(f'사용자 입력 문장과 가장 유사한 질문의 인덱스 : {idx}')

print('-'*80)

# 코사인 유사도 값이 가장 큰 질문에 대한 답변 추출
answer = df.iloc[idx, 1]
print(f'챗봇의 답변 : {answer}')

사용자 질문과 챗봇 데이터의 질문 간의 코사인 유사도 : 
[0.118883625, 0.068715364, 0.017905915, -0.05187316, 0.14834571, -0.061005358, 0.00061615993, 0.04698713, 0.078516066, 0.14175129, 0.07343854, 0.20391764, 0.14609137, 0.042446762, 0.25163916, 0.16325079, 0.030650953, 0.16306421, 0.14460745, 0.22370836, 0.23851502, 0.13288051, -0.0115006985, -0.0074457326, 0.059813976, -0.027964089, 0.07816474, 0.095666125, 0.0202023, 0.21714103, 0.10942273, 0.08108533, 0.057067115, 0.007748527, 0.041677807, 0.05737702, 0.044253685, 0.084194794, -0.015751872, 0.12353628, 0.033685986, 0.07417448, 0.036384083, 0.28648567, 0.26977244, 0.12766357, 0.035739105, -0.00436564, -0.020360969, 0.2980903, 0.22650218, 0.059527732, 0.09486984, 0.093708366, 0.07743897, 0.043464024, 0.0677497, 0.040902067, 0.11781658, 0.10654304, 0.11723778, 0.13197495, -0.074569665, 0.055987045, 0.22625442, 0.06032087, 0.071675636, 0.05356703, 0.14837861, 0.10276382, -0.0047162697, -8.6787775e-05, -0.086849935, 0.039503727, 0.15643086, 0.34322444, 0.00

In [ ]:
### 사용자 질문에 대한 응답 구현 함수 정의 → 샘플을 이용한 테스트

# 사용자 질문 생성
question = input('질문 : ')

# 사용자 질문 → 임베딩 벡터 생성
user_embedding = model.encode(question)

# 코사인 유사도 계산 (11823개(embedding)와 1개(question)의 코사인 유사도 계산 → 값을 scores에 저장 → df에 score 컬럼 생성(최대값 추출))
scores = []
for embedding in df.loc[:, 'embedding']:
  score = cos_sim(user_embedding, embedding)
  scores.append(score)

# score 컬럼 생성 → 코사인 유사도 값 저장
df.loc[:,'score'] = scores

# score 컬럼 → 값이 최대인 행 인덱스 추출(pandas.series.idxmax())
idx = df.loc[:,'score'].idxmax()

# 코사인 유사도 값이 가장 큰 질문에 대한 답변 추출
answer = df.iloc[idx, 1]
print(f'챗봇의 답변 : {answer}')

질문 : 오늘 날씨 어때
챗봇의 답변 : 좋은 사람이 찾아오려나봐요.


In [ ]:
### 사용자 질문에 대한 응답 구현 함수 정의 → "chatbot" 개발

def chatbot(question):
  # 사용자 질문 → 문장 임베딩 벡터 생성
  user_embedding = model.encode(question)
  # 사용자 질문과 챗봇 데이터 질문 간의 코사인 유사도 계산
  scores = []
  for embedding in df.loc[:, 'embedding']:
    score = cos_sim(user_embedding, embedding)
    scores.append(score)
  # score 컬럼 생성 → 코사인 유사도 값 저장(idxmax 함수가 series에 적용 가능하므로 사용을 위해서 series 생성)
  df.loc[:,'score'] = scores
  # 코사인 유사도 값 최대인 인덱스 추출 → 답변 추출
  idx = df.loc[:,'score'].idxmax()
  # 답변 추출
  answer = df.iloc[idx,1]
  return answer

In [ ]:
'''
### 사용자 질문에 대한 응답 구현 함수 정의 → "chatbot" 개발

def chatbot(question):
  user_embedding = model.encode(question)
  scores = []
  for embedding in df.loc[:, 'embedding']:
    score = cos_sim(user_embedding, embedding)
    scores.append(score)
  df.loc[:,'score'] = scores
  idx = df.loc[:,'score'].idxmax()
  answer = df.iloc[idx,1]
  return answer

  '''

## ChatBot 함수 실행

In [ ]:
### 성능 테스트(1)

# 사용자의 질문 입력
text = input("사용자 : ")

# 챗봇의 답변 출력
answer = chatbot(question = text)
print(f'chatbot : {answer}')

사용자 : 반가워
chatbot : 저도 반가워요.


In [ ]:
### 성능 테스트(2)

# 사용자 질문 입력
text = input("사용자 : ")

# 챗봇의 답변 출력
answer = chatbot(question=text)
print(f'chatbot : {answer}')

사용자 : 출근하기 싫어
chatbot : 직장 스트레스가 심한가봐요.


In [ ]:
### 성능테스트(3)

# 사용자 질문 입력
text = input("사용자 : ")

# 챗봇의 답변
answer = chatbot(question=text)
print(f'chatbot : {answer}')

사용자 : 너는 누구니
chatbot : 저는 위로해드리는 로봇이에요.


In [ ]:
### 성능테스트(4)

# 사용자 질문 입력
text = input("사용자 : ")

# 챗봇의 답변
answer = chatbot(question=text)
print(f'chatbot : {answer}')

사용자 : 몸에 열이 나고 머리가 아파요
chatbot : 감기 조심하세요!


In [ ]:
### 성능테스트(5)

# 사용자 질문 입력
text = input("사용자 : ")

# 챗봇의 답변
answer = chatbot(question=text)
print(f'chatbot : {answer}')

사용자 : 여행 가고 싶다
chatbot : 계획을 세워보세요.


In [ ]:
### 성능테스트(6)

# 사용자 질문 입력
text = input("사용자 : ")

# 챗봇의 답변
answer = chatbot(question=text)
print(f'chatbot : {answer}')

사용자 : 너무 짜증나
chatbot : 짜증날 때는 기분 푸는 음악 어때요?


In [ ]:
### 성능테스트(7)

# 사용자 질문 입력
text = input("사용자 : ")

# 챗봇의 답변
answer = chatbot(question=text)
print(f'chatbot : {answer}')

사용자 : 헤어진 남친이 너무 보고 싶어
chatbot : 헤어진 사이에 연락은 아닌 거 같아요.
